In [1]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', None)        # Let the display adjust to the window

# 2. Set the display width (optional but often helpful)
#    'None' tries to detect terminal width. 
#    A large number (e.g., 1000) ensures no wrapping unless absolutely necessary.
pd.set_option('display.width', 1000) 

In [2]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")

date_str: 2025-04-08


In [3]:
import sys
from pathlib import Path

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils

SOURCE_PATH_OHLCV = f'..\data\{date_str}_df_OHLCV_clean.parquet'
SOURCE_PATH = f'..\data\{date_str}_df_finviz_merged.parquet'
PARQUET_PATH_COVARIANCE = f'..\data\{date_str}_df_cov_emv_matrix.parquet'
PARQUET_PATH_CORRELATION = f'..\data\{date_str}_df_corr_emv_matrix.parquet'

Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src']


In [4]:
import pandas as pd

df_ohlcv = pd.read_parquet(SOURCE_PATH_OHLCV)
df_finviz = pd.read_parquet(SOURCE_PATH)

print(f"Loaded df_ohlcv shape: {df_ohlcv.shape}")
print(f"Loaded df_finviz shape: {df_finviz.shape}")


Loaded df_ohlcv shape: (381520, 9)
Loaded df_finviz shape: (1520, 55)


In [5]:
# Combine indices from both dataframes and remove duplicates
tickers = list(set(df_finviz.index))
print(f"Total unique tickers: {len(tickers)}")

Total unique tickers: 1520


In [6]:
df_close = df_ohlcv['Adj Close'].unstack(level=0)

# Assume `tickers` is your predefined list of tickers
common_tickers = df_close.columns.intersection(tickers)
df_close = df_close[common_tickers]

# Assuming `tickers` is your original list of tickers
missing_tickers = list(set(tickers) - set(df_close.columns))

# Print the missing tickers (if any)
if missing_tickers:
    print(f"{len(missing_tickers)} tickers not found in df_close:")
    print(missing_tickers)
else:
    print("All tickers in your list are present in df_close.")

print(f"\nShape of filtered df_close: {df_close.shape}")
display(df_close.head())

All tickers in your list are present in df_close.

Shape of filtered df_close: (251, 1520)


Symbol           A     AA    AAL   AAON    AAPL    ABBV    ABNB     ABT   ACGL    ACI   ACIW    ACM     ACN    ACWI    ACWV   ACWX    ADBE    ADC     ADI    ADM     ADP    ADSK   ADT    AEE   AEG    AEM    AEP    AER    AES     AFG    AFL   AFRM    AGCO    AGG    AGI  AGNC    AIG     AIT     AIZ     AJG    AKAM     ALB    ALC    ALGN     ALL    ALLE   ALLY    ALNY   ALSN     ALV     AM    AMAT  AMCR     AMD     AME    AMGN    AMH   AMLP     AMP     AMT    AMX    AMZN      AN   ANET   ANGL    ANSS     AON    AOS    APA     APD    APG    APH     APO    APP    APPF   APTV     AR   ARCC     ARE    ARES    ARGX   ARKB   ARKK     ARM   ARMK     AS    ASML    ASND     ASR  ASTS    ASX    ATI     ATO     ATR     AU   AUR     AVB   AVDE   AVDV   AVEM    AVGO   AVLV   AVTR   AVUS   AVUV     AVY     AWI     AWK    AXON     AXP    AXS   AXTA     AYI   AZEK    AZN      AZO      BA   BABA    BAC     BAH   BALL    BAM     BAP    BAX   BBAX   BBCA     BBD   BBEU   BBIN   BBIO   BBJP   BBUS   BBVA  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
2024-04-09  146.31  36.41  13.95  89.24  168.87  164.16  162.66  110.19  86.74  20.39  33.83  94.96  330.36  107.76  101.72  51.96  492.55  55.97  200.79  61.57  244.75  247.93  6.45  71.67  5.91  61.55  81.92  85.34  17.47  122.43  81.48  35.37  119.69  93.33  15.02  8.44  73.93  194.41  176.11  242.11  107.57  126.88  82.41  330.25  165.01  132.35  37.97  155.90  81.11  119.09  13.36  208.65  8.93  170.78  180.00  262.36  35.50  43.78  424.08  186.49  18.39  185.67  160.27  73.69  27.10  344.50  317.39  85.00  33.65  233.73  39.51  56.98  110.33  76.52  232.49  78.93  30.23  18.75  123.34  130.34  384.14  68.98  48.62  126.81  31.86  14.94  985.95  153.67  315.06  2.31  10.79  51.74  113.77  138.69  22.81  3.25  182.94  61.79  63.70  57.61  131.80  62.74  25.86  87.58  90.62  215.81  118.93  119.09  311.27  217.38  62.29  33.54  265.67  49.83  67.03  3043.62  178.12  71.44  36.82  141.39  66.49  40.59  167.49  41.72  46.72  65.56  2.6482  56.74  57.85  28.43  56.29  92.64  11.05   
2024-04-10  143.10  35.78  13.41  87.53  166.99  163.09  160.28  108.95  87.66  20.32  33.06  93.81  321.67  106.53  100.72  51.26  487.22  54.12  193.12  61.56  240.62  239.01  6.28  70.01  5.84  60.61  80.20  84.36  16.89  120.83  81.23  33.57  121.09  92.21  15.12  8.04  73.74  191.00  174.82  238.03  104.38  125.39  81.92  327.82  166.40  127.66  36.75  152.41  81.38  116.50  13.32  207.50  8.68  167.14  178.58  258.57  35.02  43.47  420.51  176.00  17.96  185.95  154.43  72.06  26.88  339.49  314.01  84.07  33.80  230.07  38.89  56.87  112.64  76.34  225.40  74.95  30.49  18.62  116.83  130.58  382.78  70.15  47.49  125.18  31.55  14.82  970.79  152.58  311.59  2.14  10.64  50.07  112.25  137.79  22.39  3.05  180.27  61.01  62.94  56.73  130.64  62.10  25.56  86.48  88.32  211.88  117.70  115.63  318.26  215.22  63.25  32.55  265.43  47.95  66.74  3025.75  174.63  73.00  35.77  140.61  65.90  39.03  164.68  40.91  45.79  64.60  2.5668  56.08  57.15  27.33  55.39  91.72  10.79   
2024

In [7]:
df_close_sorted = df_close.sort_index()
print(f'df_close_sorted shape: {df_close_sorted.shape}')
display(df_close_sorted.head())

# Calculate daily returns
df_close_returns = df_close_sorted.pct_change().dropna()
print(f'df_close_returns shape: {df_close_returns.shape}')
display(df_close_returns.head())


df_close_sorted shape: (251, 1520)


Symbol           A     AA    AAL   AAON    AAPL    ABBV    ABNB     ABT   ACGL    ACI   ACIW    ACM     ACN    ACWI    ACWV   ACWX    ADBE    ADC     ADI    ADM     ADP    ADSK   ADT    AEE   AEG    AEM    AEP    AER    AES     AFG    AFL   AFRM    AGCO    AGG    AGI  AGNC    AIG     AIT     AIZ     AJG    AKAM     ALB    ALC    ALGN     ALL    ALLE   ALLY    ALNY   ALSN     ALV     AM    AMAT  AMCR     AMD     AME    AMGN    AMH   AMLP     AMP     AMT    AMX    AMZN      AN   ANET   ANGL    ANSS     AON    AOS    APA     APD    APG    APH     APO    APP    APPF   APTV     AR   ARCC     ARE    ARES    ARGX   ARKB   ARKK     ARM   ARMK     AS    ASML    ASND     ASR  ASTS    ASX    ATI     ATO     ATR     AU   AUR     AVB   AVDE   AVDV   AVEM    AVGO   AVLV   AVTR   AVUS   AVUV     AVY     AWI     AWK    AXON     AXP    AXS   AXTA     AYI   AZEK    AZN      AZO      BA   BABA    BAC     BAH   BALL    BAM     BAP    BAX   BBAX   BBCA     BBD   BBEU   BBIN   BBIO   BBJP   BBUS   BBVA  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
2024-04-09  146.31  36.41  13.95  89.24  168.87  164.16  162.66  110.19  86.74  20.39  33.83  94.96  330.36  107.76  101.72  51.96  492.55  55.97  200.79  61.57  244.75  247.93  6.45  71.67  5.91  61.55  81.92  85.34  17.47  122.43  81.48  35.37  119.69  93.33  15.02  8.44  73.93  194.41  176.11  242.11  107.57  126.88  82.41  330.25  165.01  132.35  37.97  155.90  81.11  119.09  13.36  208.65  8.93  170.78  180.00  262.36  35.50  43.78  424.08  186.49  18.39  185.67  160.27  73.69  27.10  344.50  317.39  85.00  33.65  233.73  39.51  56.98  110.33  76.52  232.49  78.93  30.23  18.75  123.34  130.34  384.14  68.98  48.62  126.81  31.86  14.94  985.95  153.67  315.06  2.31  10.79  51.74  113.77  138.69  22.81  3.25  182.94  61.79  63.70  57.61  131.80  62.74  25.86  87.58  90.62  215.81  118.93  119.09  311.27  217.38  62.29  33.54  265.67  49.83  67.03  3043.62  178.12  71.44  36.82  141.39  66.49  40.59  167.49  41.72  46.72  65.56  2.6482  56.74  57.85  28.43  56.29  92.64  11.05   
2024-04-10  143.10  35.78  13.41  87.53  166.99  163.09  160.28  108.95  87.66  20.32  33.06  93.81  321.67  106.53  100.72  51.26  487.22  54.12  193.12  61.56  240.62  239.01  6.28  70.01  5.84  60.61  80.20  84.36  16.89  120.83  81.23  33.57  121.09  92.21  15.12  8.04  73.74  191.00  174.82  238.03  104.38  125.39  81.92  327.82  166.40  127.66  36.75  152.41  81.38  116.50  13.32  207.50  8.68  167.14  178.58  258.57  35.02  43.47  420.51  176.00  17.96  185.95  154.43  72.06  26.88  339.49  314.01  84.07  33.80  230.07  38.89  56.87  112.64  76.34  225.40  74.95  30.49  18.62  116.83  130.58  382.78  70.15  47.49  125.18  31.55  14.82  970.79  152.58  311.59  2.14  10.64  50.07  112.25  137.79  22.39  3.05  180.27  61.01  62.94  56.73  130.64  62.10  25.56  86.48  88.32  211.88  117.70  115.63  318.26  215.22  63.25  32.55  265.43  47.95  66.74  3025.75  174.63  73.00  35.77  140.61  65.90  39.03  164.68  40.91  45.79  64.60  2.5668  56.08  57.15  27.33  55.39  91.72  10.79   
2024

df_close_returns shape: (250, 1520)


Symbol             A        AA       AAL      AAON      AAPL      ABBV      ABNB       ABT      ACGL       ACI      ACIW       ACM       ACN      ACWI      ACWV      ACWX      ADBE       ADC       ADI       ADM       ADP      ADSK       ADT       AEE       AEG       AEM       AEP       AER       AES       AFG       AFL      AFRM      AGCO       AGG       AGI      AGNC       AIG       AIT       AIZ       AJG      AKAM       ALB       ALC      ALGN       ALL      ALLE      ALLY      ALNY      ALSN       ALV        AM      AMAT      AMCR       AMD       AME      AMGN       AMH      AMLP       AMP       AMT       AMX      AMZN        AN      ANET      ANGL      ANSS       AON       AOS       APA       APD       APG       APH       APO       APP      APPF      APTV        AR      ARCC       ARE      ARES      ARGX      ARKB      ARKK       ARM      ARMK        AS      ASML      ASND       ASR      ASTS       ASX       ATI       ATO       ATR        AU       AUR       AVB      AVDE  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
2024-04-10 -0.021940 -0.017303 -0.038710 -0.019162 -0.011133 -0.006518 -0.014632 -0.011253  0.010606 -0.003433 -0.022761 -0.012110 -0.026305 -0.011414 -0.009831 -0.013472 -0.010821 -0.033053 -0.038199 -0.000162 -0.016874 -0.035978 -0.026357 -0.023162 -0.011844 -0.015272 -0.020996 -0.011483 -0.033200 -0.013069 -0.003068 -0.050891  0.011697 -0.012000  0.006658 -0.047393 -0.002570 -0.017540 -0.007325 -0.016852 -0.029655 -0.011743 -0.005946 -0.007358  0.008424 -0.035436 -0.032131 -0.022386  0.003329 -0.021748 -0.002994 -0.005512 -0.027996 -0.021314 -0.007889 -0.014446 -0.013521 -0.007081 -0.008418 -0.056250 -0.023382  0.001508 -0.036439 -0.022120 -0.008118 -0.014543 -0.010649 -0.010941  0.004458 -0.015659 -0.015692 -0.001931  0.020937 -0.002352 -0.030496 -0.050424  0.008601 -0.006933 -0.052781  0.001841 -0.003540  0.016961 -0.023241 -0.012854 -0.009730 -0.008032 -0.015376 -0.007093 -0.011014 -0.073593 -0.013902 -0.032277 -0.013360 -0.006489 -0.018413 -0.061538 -0.014595 -0.012623   
2024-04-11  0.005800  0.001118  0.015660 -0.002856  0.043296 -0.008952  0.032069  0.002478 -0.009468 -0.008858 -0.006352  0.005543  0.002456  0.005914 -0.000894  0.002926 -0.006034  0.004065  0.011806 -0.020630 -0.005569  0.008577 -0.001592 -0.003142 -0.039384  0.006600 -0.004489 -0.011498  0.007697 -0.001655 -0.039517  0.006851 -0.027831  0.000217  0.013889  0.007463 -0.015324  0.002147 -0.021794 -0.007688  0.006802  0.020895  0.001465 -0.008907 -0.024339  0.004778  0.002993 -0.002100 -0.005161 -0.000773 -0.008258  0.017831 -0.004608  0.020103  0.004200  0.013304  0.000857 -0.000460 -0.008204 -0.009432  0.001670  0.016671 -0.006864  0.028865 -0.002232  0.001797 -0.015796  0.007256 -0.018343  0.001434  0.012600  0.009671 -0.010476  0.024889 -0.005812  0.010941 -0.010167  0.006445  0.009244  0.000230 -0.008177  0.005132  0.014951  0.047532  0.001585  0.015520  0.018027 -0.010093 -0.001508 -0.014019  0.023496  0.040344 -0.006058 -0.001887  0.052702  0.039344 -0.000832  0.000983   
2024-04-12 -0.029459 -0.

In [8]:
cov_emv_matrix, corr_emv_matrix = utils.get_cov_corr_ewm_matrices_chunked(df_close_returns, span=21, return_corr=True, return_cov=True)

print(f'cov_emv_matrix shape: {cov_emv_matrix.shape}')
display(cov_emv_matrix.head())
print(f'corr_emv_matrix shape: {corr_emv_matrix.shape}')
display(corr_emv_matrix.head())  

cov_emv_matrix shape: (1520, 1520)


Symbol         A        AA       AAL      AAON      AAPL      ABBV      ABNB       ABT      ACGL       ACI      ACIW       ACM       ACN      ACWI      ACWV      ACWX      ADBE       ADC       ADI       ADM       ADP      ADSK       ADT       AEE       AEG       AEM       AEP       AER       AES       AFG       AFL      AFRM      AGCO       AGG       AGI      AGNC       AIG       AIT       AIZ       AJG      AKAM       ALB       ALC      ALGN       ALL      ALLE      ALLY      ALNY      ALSN       ALV        AM      AMAT      AMCR       AMD       AME      AMGN       AMH      AMLP       AMP       AMT       AMX      AMZN        AN      ANET      ANGL      ANSS       AON       AOS       APA       APD       APG       APH       APO       APP      APPF      APTV        AR      ARCC       ARE      ARES      ARGX      ARKB      ARKK       ARM      ARMK        AS      ASML      ASND       ASR      ASTS       ASX       ATI       ATO       ATR        AU       AUR       AVB      AVDE      AVDV  \
Symbol                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
A       0.000554  0.000842  0.000590  0.000574  0.000592  0.000418  0.000450  0.000219  0.000270  0.000112  0.000470  0.000257  0.000379  0.000359  0.000162  0.000266  0.000322  0.000061  0.000743  0.000359  0.000271  0.000353  0.000264  0.000160  0.000474  0.000282  0.000076  0.000517  0.000550  0.000207  0.000305  0.001026  0.000499 -0.000033  0.000297  0.000179  0.000370  0.000406  0.000347  0.000221  0.000419  0.000910  0.000351  0.000427  0.000205  0.000269  0.000451  0.000644  0.000444  0.000443  0.000414  0.000626  0.000245  0.000694  0.000437  0.000195  0.000263  0.000359  0.000553  0.000049  0.000136  0.000531  0.000282  0.000742  0.000130  0.000327  0.000149  0.000224  0.001075  0.000432  0.000404  0.000567  0.000904  0.000873  0.000306  0.000538  0.000881  0.000376  0.000350  0.000954  0.000263  0.000128  0.000676  0.000761  0.000343  0.000717  0.000426  0.000317  0.000250  0.000783  0.000522  0.000763  0.000127  0.000241  0.000394  0.001057  0.000372  0.000264  0.000301   
AA      0.000842  0.001786  0.001013  0.000949  0.001095  0.000628  0.000832  0.000230  0.000403  0.000179  0.000964  0.000453  0.000626  0.000623  0.000243  0.000472  0.000599  0.000082  0.001176  0.000506  0.000418  0.000693  0.000587  0.000204  0.000843  0.000376  0.000029  0.000886  0.000959  0.000326  0.000479  0.001908  0.000964 -0.000035  0.000455  0.000300  0.000549  0.000836  0.000635  0.000309  0.000753  0.001707  0.000441  0.000851  0.000337  0.000442  0.000905  0.000902  0.000853  0.000876  0.000717  0.001058  0.000449  0.001359  0.000762  0.000223  0.000465  0.000656  0.000962  0.000016  0.000204  0.000975  0.000594  0.001236  0.000230  0.000559  0.000253  0.000416  0.002009  0.000757  0.000719  0.001015  0.001653  0.001798  0.000565  0.001062  0.001560  0.000649  0.000619  0.001707  0.000331  0.000471  0.001245  0.001279  0.000614  0.001465  0.000820  0.000564  0.000508  0.001078  0.001017  0.001362  0.000146  0.000383  0.000710  0.001845  0.000602  0.000453  0.000512   


corr_emv_matrix shape: (1520, 1520)


Symbol         A        AA       AAL      AAON      AAPL      ABBV      ABNB       ABT      ACGL       ACI      ACIW       ACM       ACN      ACWI      ACWV      ACWX      ADBE       ADC       ADI       ADM       ADP      ADSK       ADT       AEE       AEG       AEM       AEP       AER       AES       AFG       AFL      AFRM      AGCO       AGG       AGI      AGNC       AIG       AIT       AIZ       AJG      AKAM       ALB       ALC      ALGN       ALL      ALLE      ALLY      ALNY      ALSN       ALV        AM      AMAT      AMCR       AMD       AME      AMGN       AMH      AMLP       AMP       AMT       AMX      AMZN        AN      ANET      ANGL      ANSS       AON       AOS       APA       APD       APG       APH       APO       APP      APPF      APTV        AR      ARCC       ARE      ARES      ARGX      ARKB      ARKK       ARM      ARMK        AS      ASML      ASND       ASR      ASTS       ASX       ATI       ATO       ATR        AU       AUR       AVB      AVDE      AVDV  \
Symbol                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
A       1.000000  0.846690  0.676306  0.706306  0.780719  0.702698  0.661935  0.514791  0.430743  0.232387  0.711594  0.641076  0.715714  0.825006  0.491827  0.639702  0.532620  0.175724  0.863574  0.578901  0.612377  0.764487  0.522102  0.389242  0.689155  0.489502  0.187751  0.774643  0.722856  0.409983  0.539640  0.696712  0.635933 -0.303498  0.468892  0.495961  0.620170  0.638673  0.599707  0.431749  0.645041  0.809908  0.623188  0.682248  0.323954  0.655960  0.619429  0.677022  0.679019  0.742466  0.672446  0.826864  0.565509  0.784229  0.835172  0.461383  0.555251  0.728738  0.799036  0.091630  0.256251  0.763099  0.523464  0.674522  0.747806  0.657807  0.419931  0.652127  0.835991  0.821688  0.726288  0.771917  0.811607  0.471100  0.639165  0.767592  0.823557  0.705760  0.634451  0.782532  0.486034  0.160804  0.793971  0.782272  0.605860  0.692793  0.731837  0.499511  0.410574  0.591367  0.721108  0.791051  0.339873  0.524072  0.559410  0.698731  0.740862  0.598588  0.616687   
AA      0.846690  1.000000  0.646675  0.651035  0.804896  0.587815  0.681105  0.300751  0.358747  0.207503  0.812634  0.628653  0.658548  0.798796  0.410723  0.632998  0.552715  0.130920  0.762226  0.454470  0.526744  0.837035  0.647639  0.277456  0.683233  0.364451  0.039881  0.740576  0.702210  0.359769  0.472244  0.721963  0.684196 -0.181717  0.399466  0.463613  0.512864  0.732465  0.611571  0.336457  0.645494  0.846553  0.436995  0.758092  0.296502  0.601468  0.692325  0.528634  0.726776  0.817072  0.649329  0.778644  0.577030  0.855137  0.811249  0.293859  0.547016  0.742080  0.773705  0.016434  0.214193  0.779821  0.613804  0.625910  0.736230  0.627003  0.396120  0.672969  0.870145  0.803178  0.719073  0.769600  0.826773  0.540681  0.656806  0.844321  0.812351  0.678117  0.625027  0.779991  0.341576  0.329941  0.814801  0.732675  0.604216  0.788532  0.785497  0.495106  0.464496  0.453524  0.783376  0.786892  0.217428  0.464540  0.561978  0.679556  0.668680  0.571321  0.584510   


In [9]:
cov_emv_matrix.to_parquet(PARQUET_PATH_COVARIANCE, engine='pyarrow', compression='zstd')
print(f'cov_emv_matrix parqueted to {PARQUET_PATH_COVARIANCE}')
display(cov_emv_matrix.info())

corr_emv_matrix.to_parquet(PARQUET_PATH_CORRELATION, engine='pyarrow', compression='zstd')
print(f'corr_emv_matrix parqueted to {PARQUET_PATH_CORRELATION}')
display(corr_emv_matrix.info())

cov_emv_matrix parqueted to ..\data\2025-04-08_df_cov_emv_matrix.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 1520 entries, A to ZTS
Columns: 1520 entries, A to ZTS
dtypes: float64(1520)
memory usage: 17.7+ MB


None

corr_emv_matrix parqueted to ..\data\2025-04-08_df_corr_emv_matrix.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 1520 entries, A to ZTS
Columns: 1520 entries, A to ZTS
dtypes: float64(1520)
memory usage: 17.7+ MB


None